In [1]:
# Importer les modules nécessaires
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
import pickle



In [2]:
# Charger les données
try:
    data = pd.read_csv('../data/merged_69.csv', sep=";", low_memory=False)
except FileNotFoundError:
    print("Fichier introuvable")
    exit(1)

In [3]:
# Verfier la taille du dataset
print(data.shape)

(287165, 131)


In [4]:
# Definir une fonction pour identifier les colonnes avec des valeurs manquantes en pourcentage
def missing_values_table(df):
        # Total des valeurs manquantes
        mis_val = df.isnull().sum()
        # Pourcentage des valeurs manquantes
        mis_val_percent = 100 * mis_val / len(df)
        return mis_val_percent

In [5]:
# Calculer le pourcentage des valeurs manquantes
missing_values = missing_values_table(data)
print(missing_values)

Ubat_W/m²_K                          0.000348
Complément_d'adresse_bâtiment       36.268696
Méthode_application_DPE              0.000000
N°_département_(BAN)                 0.068602
Qualité_isolation_menuiseries        0.000000
                                      ...    
Emission_GES_ECS_dépensier           0.000000
Conso_5_usages_par_m²_é_primaire     0.002786
Besoin_chauffage                    11.680741
Appartement_non_visité_(0/1)        62.691136
Complément_d'adresse_logement       27.626974
Length: 131, dtype: float64


In [6]:
# Supprimer les colonnes avec plus de 30% de valeurs manquantes
data = data.drop(missing_values[missing_values > 30].index, axis=1)

In [7]:
# Supprimer les lignes qui ont moins de 9% de valeurs non manquantes
data = data.dropna(subset=missing_values[missing_values < 9].index, axis=0)

# Nouvelle taille du dataset
print(data.shape)

(220630, 103)


In [8]:
from sklearn.impute import KNNImputer

# Sélectionner les colonnes quantitatives
quantitative_columns = data.select_dtypes(include=[np.number]).columns



In [9]:
# Sélectionner les colonnes catégorielles
categorical_columns = data.select_dtypes(include=[object]).columns

# Imputer les valeurs manquantes par la mode
for col in categorical_columns:
    data[col] = data[col].fillna(data[col].mode()[0])

In [10]:
# Sélectionner les colonnes qui ne contiennent pas "Coût", "Conso" ou "GES"
variables_explicatives = [col for col in data.columns if not any(keyword in col for keyword in ["Coût", "Conso", "GES"])]


In [11]:
# Filtrer les colonnes quantitatives parmi les variables explicatives
quantitative_variables = [col for col in variables_explicatives if np.issubdtype(data[col].dtype, np.number)]


In [12]:
# Supprimer les colonnes constantes (qui ne donneront donc aucune corrélation)
quantitative_variables = [col for col in quantitative_variables if data[col].nunique() > 1]

# Calculer la corrélation entre chaque variable quantitative et 'Conso_5_usages_é_finale'
correlations = data[quantitative_variables].corrwith(data["Conso_5_usages_é_finale"])

# Trier les corrélations dans l'ordre décroissant
correlations_sorted = correlations.sort_values(ascending=False)
# Afficher les corrélations triées
#print(correlations_sorted)
# Filtrer pour garder uniquement les corrélations supérieures à 0.1
correlations_filtered = correlations_sorted[correlations_sorted > 0.1]
# Afficher les corrélations filtrées
print(correlations_filtered)

Surface_habitable_logement    0.639725
Ubat_W/m²_K                   0.314571
_score                        0.196218
Code_postal_(BAN)             0.156648
dtype: float64


In [13]:
# Decrire la variable cible
print(data['Conso_5_usages_é_finale'].describe())

count    220630.000000
mean      10416.044144
std        9504.693253
min         306.300000
25%        4985.900000
50%        7984.400000
75%       13082.150000
max      828004.300000
Name: Conso_5_usages_é_finale, dtype: float64


In [18]:
EI = 13158.900000 - 5015.400000
limitSup = 13158.900000 + (EI * 1.5)
print(limitSup) #le seuil qui considère valeurs aberrantes niv sup

# Compter le nombre de valeurs supérieures à 20000 (outliers)
count_above_20000 = (data['Conso_5_usages_é_finale'] > 25000).sum()

# Afficher le résultat
print("Nombre de valeurs supérieures à 25000 :", count_above_20000)

# Calculer le pourcentage
percentage_above_20000 = (count_above_20000 / len(data['Conso_5_usages_é_finale'])) * 100

# Afficher le résultat
print("Pourcentage de valeurs supérieures à 25000 :", percentage_above_20000, "%")

25374.15
Nombre de valeurs supérieures à 25000 : 11007
Pourcentage de valeurs supérieures à 25000 : 4.988895435797489 %


In [19]:
# Supprimer les valeurs aberrantes
data = data[data['Conso_5_usages_é_finale'] < 25000] 

In [23]:
# Type des variables
print(data.dtypes)

Ubat_W/m²_K                         float64
Méthode_application_DPE              object
N°_département_(BAN)                float64
Qualité_isolation_menuiseries        object
Nombre_appartement                  float64
                                     ...   
Nom__commune_(BAN)                   object
Emission_GES_ECS_dépensier          float64
Conso_5_usages_par_m²_é_primaire    float64
Besoin_chauffage                    float64
Complément_d'adresse_logement        object
Length: 103, dtype: object


In [ ]:
# Type des varaibles exp